In [1]:
# | default_exp _components.logger

# Logger

In [2]:
# | export

import logging
import logging.config
from typing import *

In [3]:
# | include: false

import time
import unittest

import pytest

In [4]:
# | export

# Logger Levels
# CRITICAL = 50
# ERROR = 40
# WARNING = 30
# INFO = 20
# DEBUG = 10
# NOTSET = 0

should_suppress_timestamps: bool = False


def suppress_timestamps(flag: bool = True) -> None:
    """Suppress logger timestamp

    Args:
        flag: If not set, then the default value **True** will be used to suppress the timestamp
            from the logger messages
    """
    global should_suppress_timestamps
    should_suppress_timestamps = flag


def get_default_logger_configuration(level: int = logging.INFO) -> Dict[str, Any]:
    """Return the common configurations for the logger

    Args:
        level: Logger level to set

    Returns:
        A dict with default logger configuration

    """
    global should_suppress_timestamps

    if should_suppress_timestamps:
        FORMAT = "[%(levelname)s] %(name)s: %(message)s"
    else:
        FORMAT = "%(asctime)s.%(msecs)03d [%(levelname)s] %(name)s: %(message)s"

    DATE_FMT = "%y-%m-%d %H:%M:%S"

    LOGGING_CONFIG = {
        "version": 1,
        "disable_existing_loggers": False,
        "formatters": {
            "standard": {"format": FORMAT, "datefmt": DATE_FMT},
        },
        "handlers": {
            "default": {
                "level": level,
                "formatter": "standard",
                "class": "logging.StreamHandler",
                "stream": "ext://sys.stdout",  # Default is stderr
            },
            "file": {
                "level": level,
                "formatter": "standard",
                "class": "logging.FileHandler",
                "filename": "faststream-log.txt",  # Specify the file name and path
            },
        },
        "loggers": {
            "": {"handlers": ["default", "file"], "level": level},  # root logger
        },
    }
    return LOGGING_CONFIG

Example on how to use **get_default_logger_configuration** function

In [5]:
# collapse_output

get_default_logger_configuration()

{'version': 1,
 'disable_existing_loggers': False,
 'formatters': {'standard': {'format': '%(asctime)s.%(msecs)03d [%(levelname)s] %(name)s: %(message)s',
   'datefmt': '%y-%m-%d %H:%M:%S'}},
 'handlers': {'default': {'level': 20,
   'formatter': 'standard',
   'class': 'logging.StreamHandler',
   'stream': 'ext://sys.stdout'},
  'file': {'level': 20,
   'formatter': 'standard',
   'class': 'logging.FileHandler',
   'filename': 'faststream-log.txt'}},
 'loggers': {'': {'handlers': ['default', 'file'], 'level': 20}}}

In [6]:
# | include: false

expected = {
    "version": 1,
    "disable_existing_loggers": False,
    "formatters": {
        "standard": {
            "format": "%(asctime)s.%(msecs)03d [%(levelname)s] %(name)s: %(message)s",
            "datefmt": "%y-%m-%d %H:%M:%S",
        }
    },
    "handlers": {
        "default": {
            "level": 20,
            "formatter": "standard",
            "class": "logging.StreamHandler",
            "stream": "ext://sys.stdout",
        },
        "file": {
            "level": 20,
            "formatter": "standard",
            "class": "logging.FileHandler",
            "filename": "faststream-log.txt",
        },
    },
    "loggers": {"": {"handlers": ["default", "file"], "level": 20}},
}

actual = get_default_logger_configuration()
assert actual == expected
actual

{'version': 1,
 'disable_existing_loggers': False,
 'formatters': {'standard': {'format': '%(asctime)s.%(msecs)03d [%(levelname)s] %(name)s: %(message)s',
   'datefmt': '%y-%m-%d %H:%M:%S'}},
 'handlers': {'default': {'level': 20,
   'formatter': 'standard',
   'class': 'logging.StreamHandler',
   'stream': 'ext://sys.stdout'},
  'file': {'level': 20,
   'formatter': 'standard',
   'class': 'logging.FileHandler',
   'filename': 'faststream-log.txt'}},
 'loggers': {'': {'handlers': ['default', 'file'], 'level': 20}}}

In [7]:
# | export

logger_spaces_added: List[str] = []


def get_logger(
    name: str, *, level: int = logging.INFO, add_spaces: bool = True
) -> logging.Logger:
    """Return the logger class with default logging configuration.

    Args:
        name: Pass the __name__ variable as name while calling
        level: Used to configure logging, default value `logging.INFO` logs
            info messages and up.
        add_spaces:

    Returns:
        The logging.Logger class with default/custom logging configuration

    """
    config = get_default_logger_configuration(level=level)
    logging.config.dictConfig(config)

    logger = logging.getLogger(name)
    return logger

In [8]:
# | include: false

assert type(get_logger(__name__)) == logging.Logger

with pytest.raises(TypeError) as e:
    get_logger()
assert "missing 1 required positional argument" in str(e.value)

In [9]:
logger = get_logger(__name__)
logger.info("hello")
logger = get_logger(__name__)
logger.info("hello")


def f():
    logger.info("hello")


f()

23-09-16 11:31:15.504 [INFO] __main__: hello
23-09-16 11:31:15.505 [INFO] __main__: hello
23-09-16 11:31:15.506 [INFO] __main__: hello


Example on how to use **get_logger** function

In [10]:
# collapse_output

logger = get_logger(__name__)

logger.debug("Debug")
logger.info("info")
logger.warning("Warning")
logger.error("Error")
logger.critical("Critical")

23-09-16 11:31:15.512 [INFO] __main__: info
23-09-16 11:31:15.512 [WARNING] __main__: Warning
23-09-16 11:31:15.513 [ERROR] __main__: Error
23-09-16 11:31:15.513 [CRITICAL] __main__: Critical


In [11]:
# collapse_output

suppress_timestamps()
logger = get_logger(__name__)

logger.debug("Debug")
logger.info("info")
logger.warning("Warning")
logger.error("Error")
logger.critical("Critical")

[INFO] __main__: info
[WARNING] __main__: Warning
[ERROR] __main__: Error
[CRITICAL] __main__: Critical


In [12]:
# | export


def set_level(level: int) -> None:
    """Set logger level

    Args:
        level: Logger level to set
    """
    root_logger = logging.getLogger()
    root_logger.setLevel(level)

In [13]:
level = logging.ERROR

set_level(level)

# Checking if the logger is set back to logging.INFO in dev mode
print(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == level

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

40
[ERROR] __main__: This is an error


In [14]:
# Reset log level back to INFO
level = logging.INFO

set_level(level)
logger.info("something")

[INFO] __main__: something


In [15]:
type(logging.DEBUG)

int